In [16]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from typing import Dict, Tuple, Optional, List, Union
import warnings
from dataclasses import dataclass

In [20]:
@dataclass
class RiskPrediction:
    """Container for risk prediction results."""
    wait_time: float  # Expected days between accidents
    severity_probs: Dict[int, float]  # Probability of each severity level (1-4)
    expected_severity: float  # Expected severity value
    risk_score: float  # Raw risk score
    normalized_risk: float  # Risk score normalized to 0-1
    risk_category: str  # Categorical risk level

# Road Risk Model
- Average severity multiplied by 1 over average wait time: $\frac{\mu_{\text{severity}}}{\mathbb{E}[\tau_{\text{accidents}}]}$
- Wait times estimated with Poisson regression, while severities estimated with Gamma regression.

In [26]:
class RoadRiskModel:
    """
    Advanced model for computing road risk scores combining:
    1. Poisson regression for accident frequency (wait times)
    2. Ordinal logistic regression for severity (1-4 scale)
    
    Final risk score = expected_severity / wait_time
    """
    
    RISK_CATEGORIES = {
        0.8: "Very High Risk",
        0.6: "High Risk",
        0.4: "Moderate Risk",
        0.2: "Low Risk",
        0.0: "Very Low Risk"
    }
    
    SEVERITY_LEVELS = [1, 2, 3, 4]
    
    def __init__(self, validation_fraction: float = 0.1):
        self.wait_time_model = None
        self.severity_models = None  # Will hold multiple binary logistic models
        self.scaler = StandardScaler()
        self.feature_names = None
        self.validation_fraction = validation_fraction
        self._is_fitted = False
        
    def _validate_input_data(self, X: pd.DataFrame, severities: Optional[np.ndarray] = None) -> None:
        """Validate input data format and contents."""
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Features must be provided as a pandas DataFrame")
        
        if X.empty:
            raise ValueError("Empty feature DataFrame provided")
            
        if X.isnull().all().any():
            warnings.warn("Some features contain only missing values")
            
        if severities is not None:
            unique_severities = np.unique(severities)
            if not all(sev in self.SEVERITY_LEVELS for sev in unique_severities):
                raise ValueError("Severity scores must be integers between 1 and 4")
    
    def preprocess_features(self, X: pd.DataFrame, fit: bool = True) -> np.ndarray:
        """Preprocess features with improved missing value handling and scaling."""
        self._validate_input_data(X)
        
        if fit:
            self.feature_names = X.columns.tolist()
        
        X_processed = X.copy()
        
        # Enhanced missing value handling
        for column in X_processed.columns:
            missing_mask = X_processed[column].isnull()
            if missing_mask.any():
                if X_processed[column].dtype in [np.float64, np.int64]:
                    if fit:
                        self._medians = {column: X_processed[column].median()}
                    X_processed.loc[missing_mask, column] = self._medians[column]
                else:
                    if fit:
                        self._modes = {column: X_processed[column].mode().iloc[0]}
                    X_processed.loc[missing_mask, column] = self._modes[column]
        
        # Scale features
        if fit:
            X_scaled = self.scaler.fit_transform(X_processed)
        else:
            X_scaled = self.scaler.transform(X_processed)
            
        return sm.add_constant(X_scaled)

    def _fit_ordinal_severity_model(self, X: np.ndarray, severities: np.ndarray):
        """
        Fit ordinal logistic regression for severity prediction using multiple binary logistic models.
        Uses the continuation ratio approach.
        """
        self.severity_models = {}
        
        # Fit a series of binary logistic models
        for level in self.SEVERITY_LEVELS[:-1]:
            # For each level k, model P(Y > k | Y ≥ k)
            mask = severities >= level
            if not np.any(mask):
                continue
                
            y_binary = (severities > level)[mask]
            X_subset = X[mask]
            
            model = sm.Logit(y_binary, X_subset)
            self.severity_models[level] = model.fit(disp=0)

    def fit(self, X: pd.DataFrame, wait_times: Union[np.ndarray, List[float]], 
            severities: Union[np.ndarray, List[float]]) -> Dict[str, float]:
        """
        Fit both Poisson (wait times) and ordinal logistic (severity) models.
        """
        wait_times = np.array(wait_times)
        severities = np.array(severities)
        
        self._validate_input_data(X, severities)
        
        if np.any(wait_times <= 0):
            raise ValueError("Wait times must be positive")
            
        X_processed = self.preprocess_features(X, fit=True)
        
        # Split into train/validation
        splits = train_test_split(
            X_processed, wait_times, severities,
            test_size=self.validation_fraction,
            random_state=42
        )
        X_train, X_val, wait_train, wait_val, sev_train, sev_val = splits
        
        # Fit wait time model
        self.wait_time_model = sm.GLM(
            wait_train,
            X_train,
            family=sm.families.Poisson(link=sm.families.links.log())
        )
        self.wait_time_results = self.wait_time_model.fit()
        
        # Fit ordinal severity model
        self._fit_ordinal_severity_model(X_train, sev_train)
        
        self._is_fitted = True
        
        # Calculate metrics
        metrics = self._calculate_metrics(
            X_train, X_val,
            wait_train, wait_val,
            sev_train, sev_val
        )
        
        return metrics
    
    def _predict_severity_probs(self, X: np.ndarray) -> np.ndarray:
        """
        Predict probabilities for each severity level using the continuation ratio approach.
        """
        # Initialize probability matrix
        probs = np.zeros((len(X), len(self.SEVERITY_LEVELS)))
        
        # Calculate conditional probabilities
        prev_mask = np.ones(len(X), dtype=bool)
        cumulative_prob = np.ones(len(X))
        
        for i, level in enumerate(self.SEVERITY_LEVELS[:-1]):
            if level not in self.severity_models:
                continue
                
            # P(Y > k | Y ≥ k)
            cond_prob = self.severity_models[level].predict(X[prev_mask])
            
            # P(Y = k) = P(Y ≥ k) * (1 - P(Y > k | Y ≥ k))
            probs[prev_mask, i] = cumulative_prob[prev_mask] * (1 - cond_prob)
            
            # Update for next iteration
            cumulative_prob[prev_mask] *= cond_prob
            prev_mask = cumulative_prob > 0.001  # Numerical stability threshold
        
        # Last level gets remaining probability
        probs[prev_mask, -1] = cumulative_prob[prev_mask]
        
        return probs
    
    def _calculate_metrics(self, X_train, X_val, wait_train, wait_val, 
                         sev_train, sev_val) -> Dict[str, float]:
        """Calculate comprehensive model performance metrics."""
        wait_pred_train = self.wait_time_results.predict(X_train)
        wait_pred_val = self.wait_time_results.predict(X_val)
        
        sev_probs_train = self._predict_severity_probs(X_train)
        sev_probs_val = self._predict_severity_probs(X_val)
        
        # Calculate expected severity values
        exp_sev_train = np.sum(sev_probs_train * np.array(self.SEVERITY_LEVELS), axis=1)
        exp_sev_val = np.sum(sev_probs_val * np.array(self.SEVERITY_LEVELS), axis=1)
        
        return {
            'wait_time_aic': self.wait_time_results.aic,
            'wait_time_rmse_train': np.sqrt(np.mean((wait_train - wait_pred_train) ** 2)),
            'wait_time_rmse_val': np.sqrt(np.mean((wait_val - wait_pred_val) ** 2)),
            'severity_rmse_train': np.sqrt(np.mean((sev_train - exp_sev_train) ** 2)),
            'severity_rmse_val': np.sqrt(np.mean((sev_val - exp_sev_val) ** 2)),
            'severity_accuracy_train': np.mean(np.argmax(sev_probs_train, axis=1) + 1 == sev_train),
            'severity_accuracy_val': np.mean(np.argmax(sev_probs_val, axis=1) + 1 == sev_val)
        }

    def _categorize_risk(self, normalized_score: float) -> str:
        """Convert normalized risk score to categorical risk level."""
        for threshold, category in self.RISK_CATEGORIES.items():
            if normalized_score >= threshold:
                return category
        return self.RISK_CATEGORIES[0.0]

    def predict_risk(self, X: pd.DataFrame) -> List[RiskPrediction]:
        """
        Predict comprehensive risk metrics for road segments.
        """
        if not self._is_fitted:
            raise ValueError("Model must be fitted before making predictions")
            
        X_processed = self.preprocess_features(X, fit=False)
        
        # Predict wait times
        wait_times = self.wait_time_results.predict(X_processed)
        
        # Predict severity probabilities
        severity_probs_matrix = self._predict_severity_probs(X_processed)
        
        # Calculate expected severity values
        expected_severities = np.sum(severity_probs_matrix * np.array(self.SEVERITY_LEVELS), axis=1)
        
        # Compute risk scores
        risk_scores = expected_severities / wait_times
        
        # Normalize scores
        normalized_scores = (risk_scores - risk_scores.min()) / (
            risk_scores.max() - risk_scores.min()
        )
        
        # Create prediction objects
        predictions = []
        for i in range(len(X)):
            severity_probs = dict(zip(self.SEVERITY_LEVELS, severity_probs_matrix[i]))
            pred = RiskPrediction(
                wait_time=wait_times[i],
                severity_probs=severity_probs,
                expected_severity=expected_severities[i],
                risk_score=risk_scores[i],
                normalized_risk=normalized_scores[i],
                risk_category=self._categorize_risk(normalized_scores[i])
            )
            predictions.append(pred)
            
        return predictions

    # def get_feature_importance(self) -> Dict[str, pd.DataFrame]:
    #     """Get feature importance metrics for both models."""
    #     if not self._is_fitted:
    #         raise ValueError("Model must be fitted before calculating feature importance")
        
    #     # Wait time model importance
    #     wait_conf_int = self.wait_time_results.conf_int()
    #     wait_importance = pd.DataFrame({
    #         'feature': ['const'] + self.feature_names,
    #         'coefficient': self.wait_time_results.params,
    #         'std_err': self.wait_time_results.bse,
    #         'p_value': self.wait_time_results.pvalues,
    #         'conf_int_lower': wait_conf_int[0],
    #         'conf_int_upper': wait_conf_int[1]
    #     }).sort_values('p_value')
        
    #     # Severity model importance (for each transition)
    #     severity_importance = {}
    #     for level, model in self.severity_models.items():
    #         conf_int = model.conf_int()
    #         severity_importance[f'severity_{level}'] = pd.DataFrame({
    #             'feature': ['const'] + self.feature_names,
    #             'coefficient': model.params,
    #             'std_err': model.bse,
    #             'p_value': model.pvalues,
    #             'conf_int_lower': conf_int[0],
    #             'conf_int_upper': conf_int[1]
    #         }).sort_values('p_value')
        
    #     return {
    #         'wait_time_importance': wait_importance,
    #         'severity_importance': severity_importance
    #     }

In [32]:
# Generate sample data
np.random.seed(42)
n_samples = 1000

# Create sample data
sample_data = pd.DataFrame({
    'disengagement_frequency': np.random.exponential(2, n_samples),
    'traffic_density': np.random.normal(50, 15, n_samples).clip(0, 100),
    'law_enforcement_activity': np.random.poisson(3, n_samples),
    'adverse_weather_conditions': np.random.choice([0, 1, 2, 3], n_samples),
    'crime_risk_score': np.random.normal(5, 2, n_samples).clip(0, 10),
    'pedestrian_density': np.random.exponential(10, n_samples),
    'infrastructure_quality': np.random.normal(7, 2, n_samples).clip(0, 10),
    'road_work_present': np.random.choice([0, 1], n_samples),
    'visibility_score': np.random.normal(8, 2, n_samples).clip(0, 10),
    'terrain_steepness': np.random.exponential(5, n_samples),
    'road_width': np.random.normal(12, 3, n_samples).clip(8, 24)
})

# Generate sample outcomes with integer severity
base_wait = 100
wait_times = (
    base_wait +
    -5 * sample_data['disengagement_frequency'] +
    -0.3 * sample_data['traffic_density'] +
    2 * sample_data['law_enforcement_activity'] +
    -10 * sample_data['adverse_weather_conditions'] +
    -2 * sample_data['crime_risk_score'] +
    -0.5 * sample_data['pedestrian_density'] +
    3 * sample_data['infrastructure_quality'] +
    -15 * sample_data['road_work_present'] +
    2 * sample_data['visibility_score'] +
    -1 * sample_data['terrain_steepness'] +
    0.5 * sample_data['road_width'] +
    np.random.normal(0, 5, n_samples)
)
wait_times = np.maximum(wait_times, 1)

severities = np.random.choice([1, 2, 3, 4], size=n_samples, p=[0.4, 0.3, 0.2, 0.1])

# Create and fit model
model = RoadRiskModel(validation_fraction=0.1)
metrics = model.fit(sample_data, wait_times, severities)

# Get predictions
predictions = model.predict_risk(sample_data.head())

# Print detailed results
print("\nModel Performance Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.2f}")

print("\nExample Predictions (first 5 road segments):")
for i, pred in enumerate(predictions[:5]):
    print(f"\nRoad Segment {i+1}:")
    print(f"Wait Time: {pred.wait_time:.1f} days")
    print("Severity Probabilities:")
    for level, prob in pred.severity_probs.items():
        print(f"  Level {level}: {prob:.3f}")
    print(f"Expected Severity: {pred.expected_severity:.2f}")
    print(f"Risk Score: {pred.risk_score:.3f}")
    print(f"Normalized Risk: {pred.normalized_risk:.3f}")
    print(f"Risk Category: {pred.risk_category}")

# # Get and print feature importance
# importance = model.get_feature_importance()

# print("\nWait Time Model - Feature Importance:")
# print(importance['wait_time_importance'].to_string())

# print("\nSeverity Model - Feature Importance (by threshold):")
# for level, imp_df in importance['severity_importance'].items():
#     print(f"\nSeverity Level {level}:")
#     print(imp_df.to_string())


Model Performance Metrics:
wait_time_aic: 6129.01
wait_time_rmse_train: 6.23
wait_time_rmse_val: 6.35
severity_rmse_train: 1.00
severity_rmse_val: 1.10
severity_accuracy_train: 0.40
severity_accuracy_val: 0.48

Example Predictions (first 5 road segments):

Road Segment 1:
Wait Time: 66.3 days
Severity Probabilities:
  Level 1: 0.370
  Level 2: 0.333
  Level 3: 0.172
  Level 4: 0.124
Expected Severity: 2.05
Risk Score: 0.031
Normalized Risk: 0.643
Risk Category: High Risk

Road Segment 2:
Wait Time: 56.2 days
Severity Probabilities:
  Level 1: 0.361
  Level 2: 0.270
  Level 3: 0.238
  Level 4: 0.132
Expected Severity: 2.14
Risk Score: 0.038
Normalized Risk: 1.000
Risk Category: Very High Risk

Road Segment 3:
Wait Time: 75.1 days
Severity Probabilities:
  Level 1: 0.381
  Level 2: 0.278
  Level 3: 0.241
  Level 4: 0.100
Expected Severity: 2.06
Risk Score: 0.027
Normalized Risk: 0.469
Risk Category: Moderate Risk

Road Segment 4:
Wait Time: 82.1 days
Severity Probabilities:
  Level 1: 0

/Users/aryanahri/anaconda3/envs/math4210/lib/python3.12/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(
